In [1]:
import pandas as pd
from datetime import timedelta
import random

In [2]:
col_numero_protocolo       = 'numero_protocolo'
col_data_abertura          = 'data_abertura_protocolo'
col_procedencia            = 'procedencia'
col_data_vencimento        = 'data_vencimento'
col_intervalo              = 'intervalo'
col_data_resposta          = 'data_resposta'
col_dias_ate_resposta      = 'resposta_menos_abertura'
col_dias_faltam_vencer     = 'dias_faltam_para_vencimento'
max_protocols_por_dia      = 300

In [3]:
num_protocolos = 3000
numero_protocolo = list(range(1001, 1001 + num_protocolos))

# Data de hoje
hoje = pd.Timestamp('today').normalize()

# Gera a data de abertura: garante que seja no mínimo hoje.
data_abertura_protocolo = [
    min(pd.Timestamp('2025-01-30') + pd.Timedelta(days=random.randint(0, 30)), hoje)
    for _ in range(num_protocolos)
]

num_falsos = int(num_protocolos * 0.70)
num_verdadeiros = num_protocolos - num_falsos

procedencia = [False] * num_falsos + [True] * num_verdadeiros
random.shuffle(procedencia)

# Gera a data de vencimento: a partir da data de abertura mais um período aleatório entre 10 e 14 dias.
data_vencimento = [
    data_abertura_protocolo[i] + pd.Timedelta(days=random.randint(10, 14))
    for i in range(num_protocolos)
]

# Como data_abertura é garantida >= hoje, data_vencimento será sempre > hoje.
df = pd.DataFrame({
    col_numero_protocolo: numero_protocolo,
    col_data_abertura: data_abertura_protocolo,
    col_procedencia: procedencia,
    col_data_vencimento: data_vencimento
})

df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento
0,1001,2025-02-07,False,2025-02-20
1,1002,2025-02-14,False,2025-02-25
2,1003,2025-02-16,False,2025-02-26
3,1004,2025-02-09,True,2025-02-23
4,1005,2025-02-17,True,2025-03-03
...,...,...,...,...
2995,3996,2025-02-20,False,2025-03-03
2996,3997,2025-02-20,True,2025-03-02
2997,3998,2025-02-19,True,2025-03-03
2998,3999,2025-01-30,False,2025-02-12


In [49]:
df[col_data_abertura] = pd.to_datetime(df[col_data_abertura])
df[col_data_vencimento] = pd.to_datetime(df[col_data_vencimento])

In [50]:
hoje = pd.to_datetime('today').normalize()
amanha = hoje + pd.Timedelta(days=1)
hoje, amanha

(Timestamp('2025-02-20 00:00:00'), Timestamp('2025-02-21 00:00:00'))

In [51]:
# Dicionários de contagem:
# Um para protocolos com procedencia == False
resposta_counts = {}
# Um específico para protocolos com procedencia == True
resposta_counts_true = {}

# Lista de datas disponíveis para resposta para um protocolo
def datas_disponiveis(abertura, vencimento):
    inicio = max(abertura, hoje)
    fim = vencimento
    if fim < inicio:
        return []
    return [inicio + pd.Timedelta(days=i) for i in range((fim - inicio).days + 1)]

In [52]:
df[col_dias_faltam_vencer] = (df[col_data_vencimento] - hoje).dt.days

df[col_intervalo] = df.apply(
    lambda row: (row[col_data_vencimento] - row[col_data_abertura]).days, axis=1
)
df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento,dias_faltam_para_vencimento,intervalo,data_resposta,resposta_menos_abertura,vencimento_menos_resposta
0,1001,2025-02-07,False,2025-02-20,0,13,2025-02-20,13.0,0.0
1,1002,2025-02-14,False,2025-02-25,5,11,2025-02-20,6.0,5.0
2,1003,2025-02-16,False,2025-02-26,6,10,2025-02-20,4.0,6.0
3,1004,2025-02-09,True,2025-02-23,3,14,2025-02-21,12.0,2.0
4,1005,2025-02-17,True,2025-03-03,11,14,2025-02-27,10.0,4.0
...,...,...,...,...,...,...,...,...,...
2995,3996,2025-02-20,False,2025-03-03,11,11,2025-03-03,11.0,0.0
2996,3997,2025-02-20,True,2025-03-02,10,10,2025-02-26,6.0,4.0
2997,3998,2025-02-19,True,2025-03-03,11,12,2025-03-03,12.0,0.0
2998,3999,2025-01-30,False,2025-02-12,-8,13,NaT,NaN,NaN


In [53]:
# Separar os protocolos em grupos
df_prox_venc = df[df[col_data_vencimento].isin([hoje, amanha])].copy()
df_nao_prox_venc = df[~df[col_data_vencimento].isin([hoje, amanha])].copy()

In [54]:
# Ordenando os dois grupos separadamente (procedencia True primeiro e intervalos menores primeiro)
df_prox_venc = df_prox_venc.sort_values(by=col_intervalo, ascending=True).reset_index(drop=True)
df_nao_prox_venc = df_nao_prox_venc.sort_values(by=col_intervalo, ascending=True).reset_index(drop=True)

In [55]:
import holidays
br_feriados = holidays.Brazil(years=[2025])
lista_feriados = [pd.to_datetime(data) for data in br_feriados.keys()]
def is_feriado(data):
    return data in lista_feriados or data.weekday() in [5, 6]

In [56]:
# lista_feriados.append(pd.Timestamp('2025-01-01'))

In [57]:
# Um dicionário para contar a quantidade de protocolos já alocados para cada data (para casos não prioritários)
protocolos_por_data = {}

# Função para atualizar a contagem e verificar se o limite foi atingido
def pode_alocar(data):
    return protocolos_por_data.get(data, 0) < max_protocols_por_dia

def alocar_protocolo(data):
    protocolos_por_data[data] = protocolos_por_data.get(data, 0) + 1

# Dicionário que mapeará o número do protocolo para a data de resposta final
data_resposta_dict = {}

# Processa protocolos prioritários (vencimento hoje ou amanhã)
# Aqui, pode-se usar a mesma lógica anterior ou, se necessário, ajustá-la para dar prioridade:
for idx, row in df_prox_venc.iterrows():
    disponiveis = datas_disponiveis(row[col_data_abertura], row[col_data_vencimento])
    if disponiveis:
        # Escolhe a data com menor frequência (sem restrição de quantidade máxima)
        freq = {d: resposta_counts_true.get(d, 0) for d in disponiveis}
        min_freq = min(freq.values())
        candidatas = [d for d in disponiveis if resposta_counts_true.get(d, 0) == min_freq]
        data_escolhida = min(candidatas)  # Escolhe a menor data dentre as candidatas
        resposta_counts_true[data_escolhida] = resposta_counts_true.get(data_escolhida, 0) + 1
    else:
        data_escolhida = pd.NaT

    data_resposta_dict[row[col_numero_protocolo]] = data_escolhida

# Processa protocolos NÃO prioritários, obedecendo o limite diário
for idx, row in df_nao_prox_venc.iterrows():
    disponiveis = datas_disponiveis(row[col_data_abertura], row[col_data_vencimento])
    if disponiveis:
        # Filtra apenas as datas que, somando os casos prioritários e não prioritários, ainda não ultrapassaram o limite diário
        disponiveis_validas = [
            d for d in disponiveis 
            if (resposta_counts_true.get(d, 0) + resposta_counts.get(d, 0)) < max_protocols_por_dia and not is_feriado(d)
        ]
        if disponiveis_validas:
            freq = {d: resposta_counts.get(d, 0) for d in disponiveis_validas}
            min_freq = min(freq.values())
            candidatas = [d for d in disponiveis_validas if resposta_counts.get(d, 0) == min_freq]
            data_escolhida = min(candidatas)
            resposta_counts[data_escolhida] = resposta_counts.get(data_escolhida, 0) + 1
        else:
            data_escolhida = pd.NaT  # Nenhuma data disponível sem ultrapassar o limite diário
    else:
        data_escolhida = pd.NaT

    data_resposta_dict[row[col_numero_protocolo]] = data_escolhida

# Adiciona a coluna de data de resposta ao dataframe original
df[col_data_resposta] = df[col_numero_protocolo].map(data_resposta_dict)

# Certifica que as colunas de data estão no formato datetime
df[col_data_abertura] = pd.to_datetime(df[col_data_abertura])
df[col_data_resposta] = pd.to_datetime(df[col_data_resposta])

In [58]:
df[col_data_resposta].value_counts().sort_index()

data_resposta
2025-02-20    279
2025-02-21    261
2025-02-24    172
2025-02-25    171
2025-02-26    170
2025-02-27    170
2025-02-28    169
2025-03-03    168
2025-03-04    168
2025-03-05    168
2025-03-06    167
Name: count, dtype: int64

In [59]:
procedencia_true_counts = df[df[col_procedencia] == True][col_data_resposta].value_counts().sort_index()
procedencia_true_counts

data_resposta
2025-02-20    89
2025-02-21    76
2025-02-24    53
2025-02-25    49
2025-02-26    51
2025-02-27    57
2025-02-28    55
2025-03-03    53
2025-03-04    47
2025-03-05    50
2025-03-06    52
Name: count, dtype: int64

In [60]:
# diferença
df[col_dias_ate_resposta] = (df[col_data_resposta] - df[col_data_abertura]).dt.days

# médias
media_true = df.loc[df[col_procedencia] == True, col_dias_ate_resposta].mean()
media_false = df.loc[df[col_procedencia] == False, col_dias_ate_resposta].mean()

print('média de dias procedencia true:', media_true)
print('média de dias procedencia false:', media_false)

média de dias procedencia true: 9.950949367088608
média de dias procedencia false: 10.3214535290007


In [61]:
df['vencimento_menos_resposta'] = (df[col_data_vencimento] - df[col_data_resposta]).dt.days
df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento,dias_faltam_para_vencimento,intervalo,data_resposta,resposta_menos_abertura,vencimento_menos_resposta
0,1001,2025-02-07,False,2025-02-20,0,13,2025-02-20,13.0,0.0
1,1002,2025-02-14,False,2025-02-25,5,11,2025-02-20,6.0,5.0
2,1003,2025-02-16,False,2025-02-26,6,10,2025-02-20,4.0,6.0
3,1004,2025-02-09,True,2025-02-23,3,14,2025-02-21,12.0,2.0
4,1005,2025-02-17,True,2025-03-03,11,14,2025-02-27,10.0,4.0
...,...,...,...,...,...,...,...,...,...
2995,3996,2025-02-20,False,2025-03-03,11,11,2025-03-03,11.0,0.0
2996,3997,2025-02-20,True,2025-03-02,10,10,2025-02-26,6.0,4.0
2997,3998,2025-02-19,True,2025-03-03,11,12,2025-03-03,12.0,0.0
2998,3999,2025-01-30,False,2025-02-12,-8,13,NaT,NaN,NaN


## Save table

In [62]:
df.to_csv('df.csv', index=False)

## Log

In [ ]:
import csv
import os
from datetime import datetime

arquivo_log = 'historico_log.csv'
data_execucao = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
cabecalho = ['data_execucao', col_numero_protocolo, col_data_abertura, col_procedencia, col_data_vencimento, col_data_resposta]

file_exists = os.path.exists(arquivo_log)

with open(arquivo_log, mode='a', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    if not file_exists:
        writer.writerow(cabecalho)
    
    for idx, row in df.iterrows():
        writer.writerow([
            data_execucao,
            row[col_numero_protocolo],
            row[col_data_abertura],
            row[col_procedencia],
            row[col_data_vencimento],
            row[col_data_resposta]
        ])